Run this cell to initialize population synthesis

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *


config_folder = "../configs/Chicago"
n = 500

person = process_pums_data(config_folder)
population_sample = synthesize_population(
    config_folder=config_folder, n_sample=n, min_age=16, random_state=1
)
ploc = puma_locations(config_folder)

Run this cell to initalize template generation

In [10]:
smg = SystemMessageGenerator(config_folder, "SystemMessage.j2", verbose_debug=False)
year = 2015

output = []
attribute_descriptions = get_attribute_descriptions(person)
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(system_message,"\n")


You are a 53 year old man that lives in Merrionette Park, Illinois.
You were born in the U.S. in 1962 in Illinois.
You live with an opposite-sex unmarried partner

 

You are a 37 year old woman that lives in Merrionette Park, Illinois.
You were born in the U.S. in 1978 in Illinois.
You are married  

You are a 60 year old man that lives in Chicago Ridge, Illinois.
You are not not a citizen of the U.S. and were born in 1955 in.
You are married  

You are a 74 year old man that lives in New Lenox, Illinois.
You were born in 1941 in Germany and became a U.S. citizen by naturalization in 1956.
 

You are a 56 year old man that lives in Mokena, Illinois.
You were born in the U.S. in 1959 in Illinois.
You are married  

You are a 73 year old man that lives in Frankfort Square, Illinois.
You were born in the U.S. in 1942 in Illinois.
You are married  

You are a 19 year old woman that lives in New Lenox, Illinois.
You were born in the U.S. in 1996 in Illinois.
You live with a biological son 